In [ ]:
## This code  is for the paper "Graph Neural Network based Handwritten Trajectories Recognition" submitted in ICDAR 2024

import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset
from torch_geometric.data import Data
import numpy as np 
import os
from tqdm import tqdm

print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")

class HWRDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        
        self.test = test
        self.filename = filename
        super(HWRDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        
        return self.filename

    @property
    def processed_file_names(self):
        
        self.data = pd.read_csv(self.raw_paths[0]) #.reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]
        

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0], header=None) #.reset_index()
        r1=0
        for index, row in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            n=self.data.shape[1]; x = np.zeros((n-1,1));  
            for c1 in range(n-1):
                x[c1][0] = self.data.iat[r1,c1+1]
            
            x = torch.tensor(x, dtype=torch.float)
            
            y = np.array([self.data.iat[r1,0]])   #y = self.data.iat[r1,0]
            y = torch.tensor(y, dtype=torch.long)
            
            arr1 = np.arange(0,n-2); arr2 = np.arange(1,n-1);
            edge_index = torch.tensor([arr1,arr2], dtype=torch.long)
            
            data = Data(x=x, edge_index=edge_index, y=y)
            r1 = r1+1
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))
            

    def _get_label(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))        
        return data
##
dataset= HWRDataset('data','hwr.csv')
dataset = dataset.shuffle()
##
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
##
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import GENConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GENConv(dataset.num_node_features, hidden_channels) 
        self.conv2 = GENConv(hidden_channels, hidden_channels) 
        self.conv3 = GENConv(hidden_channels, hidden_channels) 
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
         
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

##
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)  
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  
        out = model(data.x, data.edge_index, data.batch)  
        loss = criterion(out, data.y) 
        loss.backward()  
        optimizer.step()  
        optimizer.zero_grad()  

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  
        correct += int((pred == data.y).sum())  
    return correct / len(loader.dataset)  


for epoch in range(1, 1000):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')